<a href="https://colab.research.google.com/github/Nehach73/DS3001_project_voting/blob/main/Project_Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Voting

**Group 2: Randa Ampah, Neha Channamraju, Rebekah Weaver, Faith Koobial, Gian Baez, Aysha Hussen**

In [25]:
# Importing the necessary libraries
import geopandas as gpd
!pip install mapclassify
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import fiona
!pip install sklearn
from sklearn.model_selection import train_test_split

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# Loading in the data
! git clone https://github.com/Nehach73/DS3001_project_voting.git

voting = pd.read_csv('DS3001_project_voting/data/voting_VA.csv')
voting = voting.drop(columns=['Unnamed: 0', 'state', 'state_po', 'office', 'version', 'mode'], axis=1)
voting.head()

Cloning into 'DS3001_project_voting'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 137 (delta 25), reused 12 (delta 8), pack-reused 102
Receiving objects: 100% (137/137), 61.75 MiB | 12.92 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (46/46), done.


,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes
0,2000,ACCOMACK,51001,AL GORE,DEMOCRAT,5092,11925
1,2000,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352,11925
2,2000,ACCOMACK,51001,RALPH NADER,GREEN,220,11925
3,2000,ACCOMACK,51001,OTHER,OTHER,261,11925
4,2000,ALBEMARLE,51003,AL GORE,DEMOCRAT,16255,36846


In [3]:
counties = pd.read_csv('DS3001_project_voting/data/county_adjacencies.csv')
counties = counties.rename(columns={'County':'county_name', 'FIPS':'county_fips'})
counties['county_name'] = counties['county_name'].str.upper()
counties.head()

,county_name,Population2022,county_fips,District,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12
0,ACCOMACK,33191,51001,2,Northumberland,Lancaster,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALBEMARLE,114534,51003,5,CharlottesvilleCity,Nelson,Augusta,Rockingham,Greene,Orange,Louisa,Fluvanna,Buckingham,Nelson,NaN,NaN
2,ALEXANDRIACITY,155525,51510,8,Fairfax,Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALLEGHANY,14835,51005,6,Craig,Bath,Botetourt,CovingtonCity,Rockbridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMELIA,13455,51007,5,PrinceEdward,Cumberland,Powhatan,Chesterfield,Dinwiddie,Nottoway,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Combining both datasets
voting = pd.merge(voting, counties, on=['county_name','county_fips'], how='right')
voting.head()

,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes,Population2022,District,N1,...,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12
0,2000.0,ACCOMACK,51001,AL GORE,DEMOCRAT,5092.0,11925.0,33191,2,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000.0,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352.0,11925.0,33191,2,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000.0,ACCOMACK,51001,RALPH NADER,GREEN,220.0,11925.0,33191,2,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000.0,ACCOMACK,51001,OTHER,OTHER,261.0,11925.0,33191,2,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004.0,ACCOMACK,51001,JOHN KERRY,DEMOCRAT,5518.0,13356.0,33191,2,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Finding significant variables for the model
voting_vars = voting[['year', 'county_name', 'county_fips', 'Population2022', 'District']]
voting_vars = voting_vars.loc[(voting["party"] == "DEMOCRAT"),:] # removing duplicates
voting_vars.head()

# 2020 entries are showing up thrice, but every other year only shows up once
# Leaving this in could help weigh people's decision about the last election higher in our model

,year,county_name,county_fips,Population2022,District
0,2000.0,ACCOMACK,51001,33191,2
4,2004.0,ACCOMACK,51001,33191,2
7,2008.0,ACCOMACK,51001,33191,2
10,2012.0,ACCOMACK,51001,33191,2
13,2016.0,ACCOMACK,51001,33191,2


In [6]:
# defining election result as the difference between dem and rep votes
votes = pd.DataFrame()
dem_votes = voting.loc[(voting["party"] == "DEMOCRAT"),:].groupby(['county_name', 'year'])["candidatevotes"].sum()
votes['dem_votes'] = dem_votes

rep_votes = voting.loc[(voting["party"] == "REPUBLICAN"),:].groupby(['county_name', 'year'])["candidatevotes"].sum()
votes['rep_votes'] = rep_votes

voting_vars = pd.merge(voting_vars, votes, on=['county_name', 'year'], how='right')
voting_vars['result'] = voting_vars['dem_votes']-voting_vars['rep_votes']


In [7]:
voting_vars.head()

,year,county_name,county_fips,Population2022,District,dem_votes,rep_votes,result
0,2000.0,ACCOMACK,51001,33191,2,5092.0,6352.0,-1260.0
1,2004.0,ACCOMACK,51001,33191,2,5518.0,7726.0,-2208.0
2,2008.0,ACCOMACK,51001,33191,2,7607.0,7833.0,-226.0
3,2012.0,ACCOMACK,51001,33191,2,7655.0,8213.0,-558.0
4,2016.0,ACCOMACK,51001,33191,2,6740.0,8583.0,-1843.0


In [8]:
voting_vars['year_code'] = 2000
voting_vars.loc[((voting_vars['year']>=2008) & (voting_vars['year']<=2012)),'year_code']= 125
voting_vars.loc[((voting_vars['year']>=2015) & (voting_vars['year']<=2020)) ,'year_code'] = 195

In [9]:
voting_vars.head()

,year,county_name,county_fips,Population2022,District,dem_votes,rep_votes,result,year_code
0,2000.0,ACCOMACK,51001,33191,2,5092.0,6352.0,-1260.0,2000
1,2004.0,ACCOMACK,51001,33191,2,5518.0,7726.0,-2208.0,2000
2,2008.0,ACCOMACK,51001,33191,2,7607.0,7833.0,-226.0,125
3,2012.0,ACCOMACK,51001,33191,2,7655.0,8213.0,-558.0,125
4,2016.0,ACCOMACK,51001,33191,2,6740.0,8583.0,-1843.0,195


In [10]:
melted_df = pd.read_csv('/content/DS3001_project_voting/data/df_melt.csv')
melted_df = melted_df.rename(columns={'FIPS':'county_fips',"YEAR":'year_code'})

voting_vars = pd.merge(voting_vars, melted_df, on=['county_fips','year_code'], how='right')
voting_vars.head()

,year,county_name,county_fips,Population2022,District,dem_votes,rep_votes,result,year_code,AV0AA,...,A88AD,A88AE,AB2AA,BD5AA,AX6AA,CL6AA,AX7AA,AX7AB,BV8AA,BV8AB
0,2000.0,ACCOMACK,51001,33191.0,2.0,5092.0,6352.0,-1260.0,2000,38305.0,...,3781.0,3195.0,34821.0,16309.0,37715.0,6788.0,6788.0,30927.0,957.0,5291.0
1,2004.0,ACCOMACK,51001,33191.0,2.0,5518.0,7726.0,-2208.0,2000,38305.0,...,3781.0,3195.0,34821.0,16309.0,37715.0,6788.0,6788.0,30927.0,957.0,5291.0
2,2008.0,ACCOMACK,51001,33191.0,2.0,7607.0,7833.0,-226.0,125,33454.0,...,2649.0,4604.0,50250.0,22909.0,33012.0,6678.0,6678.0,26334.0,1002.0,5147.0
3,2012.0,ACCOMACK,51001,33191.0,2.0,7655.0,8213.0,-558.0,125,33454.0,...,2649.0,4604.0,50250.0,22909.0,33012.0,6678.0,6678.0,26334.0,1002.0,5147.0
4,2016.0,ACCOMACK,51001,33191.0,2.0,6740.0,8583.0,-1843.0,195,32673.0,...,2395.0,5072.0,57711.0,26018.0,32325.0,6141.0,6141.0,26184.0,677.0,6603.0


In [11]:
# prompt: Using dataframe voting_vars: time series matplot with different colors for each line

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 5))

colors = ['red', 'green', 'blue', 'yellow', 'purple']

for i, year in enumerate(voting_vars['year'].unique()):
    df_year = voting_vars[voting_vars['year'] == year]
    ax.plot(df_year['county_name'], df_year['result'], color=colors[i])

ax.set_xlabel('County Name')
ax.set_ylabel('Result')
ax.set_title('

SyntaxError: unterminated string literal (detected at line 15) (<ipython-input-11-0cde1d8e75e8>, line 15)

In [ ]:
import altair as alt
alt.Chart(voting_vars).mark_line().encode(
    alt.X('year:T'),
    alt.Y('result:Q'),
    color='county_name'
).properties(width=800, height=500)

In [ ]:
plt.plot(voting_vars['year'], voting_vars['result'])
plt.xlabel('Year')
plt.ylabel("Result")
plt.show()

In [ ]:
# Cleaning votes for map
#voting_2022 = voting.loc[voting["year"] == 2020,:] #collect 2022 data

#dem_voting_2022 = voting_2022.loc[(voting_2022["party"] == "DEMOCRAT"),:].groupby("county_fips")["candidatevotes"].sum()
#dem_voting_2022 = dem_voting_2022.rename("dem_votes_2022")
#df = voting.merge(dem_voting_2022, left_on="Population2022", right_on="county_fips")

#rep_voting_2022 = voting_2022.loc[(voting_2022["party"] == "REPUBLICAN"),:].groupby("county_fips")["candidatevotes"].sum()
#rep_voting_2022 = rep_voting_2022.rename("rep_votes_2022")

#df = df.merge(rep_voting_2022, left_on="Population2022", right_on="county_fips")

In [ ]:
#fig = plt.figure()
#ax = fig.add_axes([0, 0, 2, 2])
#divider = make_axes_locatable(ax)
#ax.axis('off')
#cax = divider.append_axes("right", size="2%", pad=0.08)
#voting_2022.plot(ax=ax,column='Population2022',legend=True)

In [28]:
y = voting_vars['result']
voting_vars['year_county'] = voting_vars['year'].astype('str')+'X'+voting_vars['county_name'].astype('str')
X = voting_vars.loc[:,['dem_votes', 'rep_votes'] ]
X_train, X_test, y_train, y_test = train_test_split(X,y, # Feature and target variables
                                                    test_size=.2, # Split the sample 80 train/ 20 test
                                                    random_state=100) # For replication purposes

Z_train = pd.concat([X_train['dem_votes'],
                    pd.get_dummies(X_train['year_county'], dtype='int')],
                    axis = 1)
Z_test = pd.concat([X_test['dem_votes'],
                    pd.get_dummies(X_test['year_county'], dtype='int')],
                    axis = 1)

reg = linear_model.LinearRegression(fit_intercept=False).fit(Z_train,y_train) # Run regression

y_hat = reg.predict(Z_test)
print('Rsq: ', reg.score(Z_test,y_test)) # R2
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse) # R2
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
results

KeyError: 'year_county'